so my plan is that i would have 2 different iterations of the dataset: one iteration would be the raw dataset. The second iteration would have the resultant vector from every consecutive frame that would provide us with the vector values of the movement of the observed landmark. Now both of the datasets would be reshapes to better resemble an image even if they are far from an recognisable image. If the resizing is not feasable, we would transform the "images" into a uniform shape so that it could pass the model. 

After that is done, i would create a model with 2 convolutional branches that woudl recognize teh different characteristics of the "images", and whatever features it was able to extract from the "images" woould be utilized in the final prediction process as the two braches merge into each other(note that the vanilla version of the dataset brach would be be independant, but the other branch would be heavily influenced by the vanilla branch. This is done so that landmark movement vector has a reference point in space. 

The outputs of the model, both for training and validation, would also be transformed into image-like state so as to simplify the prediction process(note that since not all of the clips do not have the same frames, there woudl be a few null values, and to take care of those values we would just replace those values with 0. This is done because the data we are using are spatial data, and any misrepresentation could drastically affect the prediction of the model.

Once we have the prediction, we woudld flatten out the data and then create another model, but this time it would be different, as i would be making a transfomer neural network. At the point of writting this manifesto, i have very little knowledge about transformer neural networks, so once i have gathered sufficient data regarding these model- I will be optimistic and vague.

In [409]:
import torch
from torch import nn
from torch.utils import data
import torchmetrics
from sklearn.model_selection import train_test_split
import glob
from tqdm import notebook, tqdm
from tqdm.notebook import tnrange
from typing import List, Tuple, Dict
from collections import defaultdict
import pandas as pd
import numpy as np
import logging as lg
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import json

In [32]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Available device: {device}")

Available device: mps


In [310]:
df = pd.read_parquet("/Users/arvinprince/pytorch-files/Google ASL/data/296317215.parquet").reset_index()

In [261]:
with open("/Volumes/ExternalHDD/asl-fingerspelling/character_to_prediction_index.json") as json_file:
    CHAR2ORD = json.load(json_file)
# CHAR2ORD = {j:i for i,j in CHAR2ORD.items()}

In [253]:
# display(pd.Series(CHAR2ORD).to_frame("Ordinal Encoding"))

In [311]:
print(f"Shape of the dataset: {df.shape}")
df.head()

Shape of the dataset: (165396, 1631)


,sequence_id,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,1494438637,0,0.558159,0.550603,0.554162,0.543828,0.551478,0.555103,0.566358,0.455327,...,-0.124966,-0.113680,-0.103391,-0.118710,-0.091068,-0.071286,-0.095167,-0.091655,-0.064835,-0.046677
1,1494438637,1,0.559157,0.550627,0.554241,0.543452,0.551426,0.554883,0.565622,0.455519,...,-0.101074,-0.092762,-0.083472,-0.102874,-0.077500,-0.058431,-0.076877,-0.079398,-0.056055,-0.039821
2,1494438637,2,0.555381,0.551183,0.554637,0.543676,0.551955,0.555277,0.565502,0.454227,...,-0.139371,-0.124513,-0.095812,-0.131984,-0.104427,-0.075522,-0.096968,-0.116063,-0.090169,-0.064778
3,1494438637,3,0.557249,0.550174,0.553681,0.542862,0.550985,0.554343,0.564671,0.454935,...,-0.187725,-0.179453,-0.133177,-0.173193,-0.143141,-0.115510,-0.135543,-0.150484,-0.123912,-0.102089
4,1494438637,4,0.557520,0.551252,0.554584,0.543844,0.552108,0.555526,0.566051,0.454531,...,-0.167483,-0.152804,-0.127324,-0.171816,-0.133314,-0.097499,-0.145111,-0.160346,-0.122395,-0.090545


468 face 25 pose 21 right 21 left


In [312]:
# filling the nan values with 0
df.fillna(0, inplace=True)

# dropping z values from the dataset 
df.drop([col for col in df.columns if "z" in col], axis = 1, inplace=True)
# dropping the pose values below the waist
df.drop([col for col in df.columns if "pose" in col and int(col[7:]) >= 25], axis = 1, inplace=True)
# saving the sequence id and the frame values in another variable
metadata= df.iloc[:,:2]
# # we are taking the Euclidian norm or L2 norm of the vectors 

# taking the difference of every two consective frame
df = df.iloc[:,2:].diff()
# dropping the nan values, specifically the first row, after taking the difference
df.dropna(inplace=True)
# squaring every value in the dataframe
df = df.apply(lambda x: x**2)
# taking the sum of the coordinate of every single landmark basically taking the modulus of the vector
landmarks = ["_face_", "_pose_", "_right_hand_", "_left_hand_"]
for l in landmarks:
    if l == landmarks[0]:
        for n in tnrange(468):
            df["y"+l+str(n)] = df["x"+l+str(n)] + df["y"+l+str(n)]
    if l == landmarks[1]:
        for n in tnrange(25):
            df["y"+l+str(n)] = df["x"+l+str(n)] + df["y"+l+str(n)]
    if l == landmarks[2]:
        for n in tnrange(21):
            df["y"+l+str(n)] = df["x"+l+str(n)] + df["y"+l+str(n)]
    if l == landmarks[3]:
        for n in tnrange(21):
            df["y"+l+str(n)] = df["x"+l+str(n)] + df["y"+l+str(n)]
# dropping off the x values since we took the sum on the y values
df.drop([x for x in df.columns if "x" in x], inplace= True, axis = 1)
# concatenating the metadata to the processed dataset
df = pd.concat([metadata,df.apply(lambda x: x**0.5)], axis=1)
# removing the nan values due to adding the metadata
df = df.dropna()
# removing the first frame of every sequence id because of spilling of the previous sequence id's last frame into the first frame of the particular sequence if
df.drop(index=list(df.query("frame == 0").index), inplace=True)
print(f"Shape of the dataset after processing it {df.shape}")
df.head()

  0%|          | 0/468 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Shape of the dataset after processing it (164396, 537)


,sequence_id,frame,y_face_0,y_face_1,y_face_2,y_face_3,y_face_4,y_face_5,y_face_6,y_face_7,...,y_right_hand_11,y_right_hand_12,y_right_hand_13,y_right_hand_14,y_right_hand_15,y_right_hand_16,y_right_hand_17,y_right_hand_18,y_right_hand_19,y_right_hand_20
1,1494438637,1,0.001591,0.000172,0.000214,0.000375,0.000112,0.000222,0.000758,0.000745,...,0.051241,0.042954,0.055037,0.059306,0.062434,0.058445,0.062547,0.066954,0.068401,0.065617
2,1494438637,2,0.004113,0.001664,0.001476,0.001268,0.001613,0.001450,0.000986,0.001597,...,0.041070,0.049006,0.070630,0.062432,0.057434,0.054491,0.089510,0.083886,0.076243,0.077719
3,1494438637,3,0.002160,0.001011,0.000960,0.000813,0.000972,0.000934,0.000831,0.001543,...,0.025862,0.029043,0.054162,0.031141,0.022392,0.023426,0.058553,0.036993,0.026859,0.027531
4,1494438637,4,0.000711,0.002018,0.001578,0.001904,0.002102,0.002106,0.002063,0.000520,...,0.030526,0.035471,0.009824,0.024465,0.039194,0.040566,0.014380,0.031524,0.036001,0.037391
5,1494438637,5,0.001263,0.002059,0.001712,0.001721,0.002088,0.001934,0.001367,0.001977,...,0.010642,0.005636,0.056476,0.031731,0.026046,0.020409,0.067420,0.062809,0.045857,0.033927


<b><h2>the table above has the optical flow values of every significant landmark</h2>

In [38]:
train_csv.head()

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


In [39]:
train_csv.nunique()

path                 68
file_id              68
sequence_id       67208
participant_id       94
phrase            46478
dtype: int64

In [21]:
67208/68

988.3529411764706

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164396 entries, 1 to 165395
Columns: 537 entries, sequence_id to y_right_hand_20
dtypes: float32(535), int16(1), int64(1)
memory usage: 338.3 MB


In [117]:
400*68/1000

27.2

In [120]:
df.frame.max()

664

In [129]:
sequence_id = list(df.drop("frame", axis=1).groupby("sequence_id").grouper)

In [193]:
X = []
coords = df.groupby("sequence_id").get_group(sequence_id[1])
coords1 = df.groupby("sequence_id").get_group(sequence_id[2])
coords = np.asarray(coords.drop(["sequence_id", "frame"], axis= 1))
coords1 = np.asarray(coords1.drop(["sequence_id", "frame"], axis= 1))
# np.save("/Volumes/ExternalHDD/asl-fingerspelling/np.npy", l)

In [200]:
coords.transpose().shape, coords1.transpose().shape

((535, 97), (535, 184))

rather than just using the cartesian coordinates that have been provided we can use the resultant vector from consecutive frames to track the movement of the arms the face and the posture. Using this

In [432]:
np.array(df.groupby("sequence_id").get_group(1494438637).iloc[:, 2:], dtype=np.float16)

array([[0.001591 , 0.0001718, 0.0002141, ..., 0.06696  , 0.0684   ,
        0.0656   ],
       [0.004112 , 0.001664 , 0.001476 , ..., 0.08386  , 0.07623  ,
        0.0777   ],
       [0.002161 , 0.001011 , 0.00096  , ..., 0.037    , 0.02686  ,
        0.02753  ],
       ...,
       [0.001054 , 0.003738 , 0.003016 , ..., 0.003862 , 0.006214 ,
        0.005867 ],
       [0.001858 , 0.0004613, 0.0006757, ..., 0.001251 , 0.00578  ,
        0.00954  ],
       [0.001436 , 0.003063 , 0.002632 , ..., 0.0114   , 0.01354  ,
        0.02129  ]], dtype=float16)

In [435]:
np.array(df.groupby("sequence_id").get_group(1494438637).iloc[:, 2:], dtype=np.float16).reshape(())

ValueError: cannot reshape array of size 56710 into shape ()

In [ ]:
56710

In [272]:
train_csv.nunique()

path                 68
file_id              68
sequence_id       67208
participant_id       94
phrase            46478
dtype: int64

In [274]:
CHAR2ORD.keys()

dict_keys([' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~'])

In [287]:
# bar = notebook(target=train_)
char = []
for n, i in enumerate(train_csv.phrase[0]):
    char.append(CHAR2ORD[i])
np.array(char)

array([18,  0, 34, 49, 36, 36, 42, 39, 46, 52, 50, 36])

In [282]:
train_csv.phrase[0]

'3 creekhouse'

In [425]:
train_csv = pd.read_csv("/Users/arvinprince/pytorch-files/Google ASL/data/train.csv")
sequence_ids = df.sequence_id.unique()
train_csv.set_index("sequence_id", inplace=True)
# phrases = train_csv.loc[sequence_ids, "phrase"]
phrases = pd.DataFrame(train_csv.loc[sequence_ids, ["phrase"]])
# train_csv

In [426]:
def phrase2array(string:str):
    arr = []
    for char in string:
        arr.append(CHAR2ORD[char])
    return np.asarray(arr, dtype= np.float16)

In [427]:
def sequence_id2optical_flow(s_ids:List):
    X = []
    for ids in s_ids:
        X.append(np.asarray(df.groupby("sequence_id").get_group(ids).iloc[:,2:], dtype=np.float16))
    return np.array(X, dtype="object")

In [428]:
# phrases.reset_index()
phrases = phrases.phrase.apply(phrase2array)

In [429]:
optical_flow = pd.Series(sequence_id2optical_flow(list(phrases.index)), name="optical_flow")

In [430]:
dataset = pd.concat([phrases.reset_index(), optical_flow], axis=1)

In [431]:
dataset.drop(["sequence_id"], axis=1, inplace=True)
dataset.head()

,phrase,optical_flow
0,"[38.0, 43.0, 32.0, 35.0, 56.0, 50.0, 0.0, 38.0...","[[0.001591, 0.0001718, 0.0002141, 0.0003755, 0..."
1,"[23.0, 22.0, 23.0, 15.0, 0.0, 46.0, 43.0, 35.0...","[[0.00594, 0.004566, 0.00405, 0.003918, 0.0046..."
2,"[10.0, 21.0, 17.0, 12.0, 23.0, 22.0, 24.0, 12....","[[0.01012, 0.01118, 0.008064, 0.004234, 0.0100..."
3,"[54.0, 54.0, 54.0, 13.0, 34.0, 39.0, 40.0, 45....","[[0.002455, 0.004124, 0.0047, 0.00502, 0.00409..."
4,"[22.0, 21.0, 19.0, 17.0, 0.0, 50.0, 46.0, 52.0...","[[0.0008645, 0.002289, 0.002016, 0.0021, 0.002..."


In [405]:
dataset_np = dataset.to_numpy().transpose()

In [403]:
np.save("/Volumes/ExternalHDD/asl-fingerspelling/X_np.npy",dataset.to_numpy().transpose())

In [406]:
X, y = dataset_np[1], dataset_np[0]

In [415]:
model = keras.Sequential()
model.add(layers.LSTM(100))
model.add(layers.LSTM(64))
model.add(layers.LSTM(64))
model.add(layers.Dense(128))
model.add(layers.Dense(128))
model.add(layers.Dense(1))

In [416]:
model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.01),
             loss=keras.losses.CategoricalCrossentropy())

In [417]:
model.fit(x=X.astype(np.float32), 
         y=y.astype(np.float32))

ValueError: setting an array element with a sequence.

Rather than pushing the dataset directly into the model, try to emulate what you did with images i.e. images would be interpreted as images- in that same way feed the optical flow data into the model reshaping the input and output data as images and the later on reshape them into a straight line to later process it through a transformer model or a LSTM  